**Curso de Inteligencia Artificial y Aprendizaje Profundo**


# Long Time Series

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Contenido

* [Introducción](#Introducción)
* [Importar las librería requeridas](#Importar-las-librería-requeridas)
* [Conjunto de datos household power consumption](#Conjunto-de-datos-household-power-consumption)
* [Preprocesamiento](#Preprocesamiento)
* [Clase create_ts_files](#Clase-create_ts_files)
* [Clase TimeSeriesLoader](#Clase-TimeSeriesLoader)
* [Crea el modelo LSTM](#Crea-el-modelo-LSTM)
* [Entrenamiento](#Entrenamiento)
* [Evaluación del modelo](#Evaluación-del-modelo)

## Referencias

1. [Introducción a Redes LSTM](Intro_LSTM.ipynb)
2. [Time Series Forecasting with LSTMs using TensorFlow 2 and Keras in Python](https://towardsdatascience.com/time-series-forecasting-with-lstms-using-tensorflow-2-and-keras-in-python-6ceee9c6c651/)
3. [Tensoflow-Time series forecasting](https://www.tensorflow.org/tutorials/structured_data/time_series)
4. Adaptado de [3 Steps to Time Series Forecasting: LSTM with TensorFlow Keras
](https://www.justintodata.com/forecast-time-series-lstm-with-tensorflow-keras/)

## Introducción

El conjunto de datos que vamos a utilizar es el consumo de energía eléctrica de los hogares de Kaggle. 

Proporciona mediciones del consumo de energía eléctrica en un hogar con una frecuencia de muestreo de un minuto.


## Importar las librerías requeridas

In [11]:
# import packages
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import time

import os

## Conjunto de datos household power consumption

Hay 2.075.259 mediciones recopiladas en 4 años. Están disponibles diferentes magnitudes eléctricas y algunos valores de submedición. 

Nos centraremos en tres variables (features):

- Fecha: fecha en formato dd / mm / aaaa
- Hora: hora en formato hh: mm: ss
- Potencia_activa_global: potencia activa promedio por minuto global del hogar (en kilovatios)


En esta lección, predeciremos la cantidad de Global_active_power con 10 minutos de anticipación.

Hemos bajado los datos y los tenemos localmente.

In [12]:
# read the dataset into python
df = pd.read_csv('../Datos/household_power_consumption.txt', delimiter=';')
df.head()

/home/alvaro/anaconda3/envs/tf2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [ ]:
df.shape

## Preprocesamiento

Transformamos el conjunto de datos df así:

- Creando la función *date_time* en formato *DateTime* combinando Fecha y Hora.
- Convertiendo *Global_active_power* a numérico y eliminar los valores faltantes (1,25%).
- Ordenando las variables por tiempo en el nuevo conjunto de datos.

In [13]:
%%time

# This code is copied from https://towardsdatascience.com/time-series-analysis-visualization-forecasting-with-lstm-77a905180eba
# with a few minor changes.
#
df['date_time'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df['Global_active_power'] = pd.to_numeric(df['Global_active_power'], errors='coerce')
df = df.dropna(subset=['Global_active_power'])

df['date_time'] = pd.to_datetime(df['date_time'])

df = df.loc[:, ['date_time', 'Global_active_power']]
df.sort_values('date_time', inplace=True, ascending=True)
df = df.reset_index(drop=True)

print('Number of rows and columns after removing missing values:', df.shape)
print('The time series starts from: ', df['date_time'].min())
print('The time series ends on: ', df['date_time'].max())

Number of rows and columns after removing missing values: (2049280, 2)
The time series starts from:  2006-12-16 17:24:00
The time series ends on:  2010-12-11 23:59:00
CPU times: user 5min 15s, sys: 620 ms, total: 5min 16s
Wall time: 5min 17s


In [15]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049280 entries, 0 to 2049279
Data columns (total 2 columns):
 #   Column               Dtype         
---  ------               -----         
 0   date_time            datetime64[ns]
 1   Global_active_power  float32       
dtypes: datetime64[ns](1), float32(1)
memory usage: 23.5 MB


,date_time,Global_active_power
0,2006-12-16 17:24:00,4.216
1,2006-12-16 17:25:00,5.360
2,2006-12-16 17:26:00,5.374
3,2006-12-16 17:27:00,5.388
4,2006-12-16 17:28:00,3.666
5,2006-12-16 17:29:00,3.520
6,2006-12-16 17:30:00,3.702
7,2006-12-16 17:31:00,3.700
8,2006-12-16 17:32:00,3.668
9,2006-12-16 17:33:00,3.662


In [16]:
df = df.astype({"Global_active_power": 'float32'})

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049280 entries, 0 to 2049279
Data columns (total 2 columns):
 #   Column               Dtype         
---  ------               -----         
 0   date_time            datetime64[ns]
 1   Global_active_power  float32       
dtypes: datetime64[ns](1), float32(1)
memory usage: 23.5 MB


### Datos de entrenamiento, validación y prueba

A continuación, dividimos el conjunto de datos en conjuntos de datos de *entrenamiento, validación y prueba*.


- *df_test* contiene los datos de los últimos 7 días en el conjunto de datos original. 
- *df_val* tiene datos 14 días antes del conjunto de datos de prueba. 
- *df_train* tiene el resto de los datos.

In [ ]:
# Split into training, validation and test datasets.
# Since it's timeseries we should do it by date.
test_cutoff_date = df['date_time'].max() - timedelta(days=7)
val_cutoff_date = test_cutoff_date - timedelta(days=14)

df_test = df[df['date_time'] > test_cutoff_date]
df_val = df[(df['date_time'] > val_cutoff_date) & (df['date_time'] <= test_cutoff_date)]
df_train = df[df['date_time'] <= val_cutoff_date]

#check out the datasets
print('Test dates: {} to {}'.format(df_test['date_time'].min(), df_test['date_time'].max()))
print('Validation dates: {} to {}'.format(df_val['date_time'].min(), df_val['date_time'].max()))
print('Train dates: {} to {}'.format(df_train['date_time'].min(), df_train['date_time'].max()))

### Transformacion de datos para Tensorflow

División del conjunto de datos en marcos de datos más pequeños

Como se mencionó anteriormente, queremos pronosticar el Global_active_power que será de 10 minutos en el futuro.

El siguiente gráfico visualiza el problema: usar los datos rezagados de $(t-n($ a $(t-1)$ para predecir el objetivo $(t + 10)$.

<figure>
<center>
<img src="../Imagenes/image-11.png" width="600" height="600" align="center"/>
</center>
</figure>

Imagen tomada de: [3 Steps to Time Series Forecasting: LSTM with TensorFlow Keras
](https://www.justintodata.com/forecast-time-series-lstm-with-tensorflow-keras/)

No es eficiente recorrer el conjunto de datos mientras se entrena el modelo. Además tensorflow espera los datos de la forma como vamos a organizarlos.

Por eso, queremos transformar el conjunto de datos con cada fila que representa los datos históricos y el objetivo.

<figure>
<center>
<img src="../Imagenes/matrix-10.png" width="400" height="300" align="center"/>
</center>
</figure>

Imagen tomada de: [3 Steps to Time Series Forecasting: LSTM with TensorFlow Keras
](https://www.justintodata.com/forecast-time-series-lstm-with-tensorflow-keras/)

De esta manera, solo necesitamos entrenar el modelo usando cada fila de la matriz anterior.

Ahora aquí vienen los desafíos:

- ¿Cómo convertimos el conjunto de datos a la nueva estructura?
- ¿Cómo manejamos esta nueva estructura de datos más grande cuando la memoria de nuestra computadora es limitada?


Como resultado, se define la función *create_ts_files*:

- para convertir el conjunto de datos original en el nuevo conjunto de datos anterior.
- al mismo tiempo, para dividir el nuevo conjunto de datos en archivos más pequeños, lo que es más fácil de procesar.

## Clase create_ts_files

Dentro de esta función, definimos los siguientes parámetros:

- *start_index*: el momento más temprano en ser incluido en todos los datos históricos para la previsión.
En esta práctica, queremos incluir el historial desde el principio, por lo que establecemos el valor predeterminado en 0.
- end_index: el último tiempo que se incluirá en todos los datos históricos para la previsión.
En esta práctica, queremos incluir todo el historial, por lo que establecemos el valor predeterminado en Ninguno.
- *history_length*: esto se menciona anteriormente, que es el número de pasos de tiempo para mirar hacia atrás para cada pronóstico.
- *step_size*: el paso de la ventana del historial.
- *Global_active_power* no cambia rápidamente a lo largo del tiempo. Entonces, para ser más eficientes, podemos dejar step_size = 10. De esta manera, reducimos la muestra para usar cada 10 minutos de datos en el pasado para predecir la cantidad futura. Solo estamos mirando t-1, t-11, t-21 hasta t-n para predecir t + 10.
- *target_step*: el número de períodos en el futuro para predecir.
Como se mencionó anteriormente, estamos tratando de predecir la potencia_activa global con 10 minutos de anticipación. Entonces esta característica = 10.
- *num_rows_per_file*: el número de registros para poner en cada archivo.


Esto es necesario para dividir el nuevo conjunto de datos grande en archivos más pequeños.
carpeta_datos: la única carpeta que contendrá todos los archivos.


Al final, solo se necesita saber que esta función crea una carpeta con archivos y que cada archivo contiene un dataframe de pandas que se parece al nuevo conjunto de datos del gráfico anterior.
Cada uno de estos dataframe tiene columnas:

- $y$, que es el objetivo a predecir. Este será el valor en *t + target_step* ($t + 10$).
- *x_lag {i}*, el valor en el tiempo *t + target_step - i* ($t + 10 - 11$, $t + 10 - 21$, y así sucesivamente), es decir, el valor rezagado en comparación con $y$.

Al mismo tiempo, la función también devuelve el número de retrasos (len (col_names) -1) en los dataframes. Este número será necesario para definir la forma de los modelos de TensorFlow más adelante.

In [ ]:
# Goal of the model:
#  Predict Global_active_power at a specified time in the future.
#   Eg. We want to predict how much Global_active_power will be ten minutes from now.
#       We can use all the values from t-1, t-2, t-3, .... t-history_length to predict t+10


def create_ts_files(dataset, 
                    start_index, 
                    end_index, 
                    history_length, 
                    step_size, 
                    target_step, 
                    num_rows_per_file, 
                    data_folder):
    assert step_size > 0
    assert start_index >= 0
    
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    
    time_lags = sorted(range(target_step+1, target_step+history_length+1, step_size), reverse=True)
    col_names = [f'x_lag{i}' for i in time_lags] + ['y']
    start_index = start_index + history_length
    if end_index is None:
        end_index = len(dataset) - target_step
    
    rng = range(start_index, end_index)
    num_rows = len(rng)
    num_files = math.ceil(num_rows/num_rows_per_file)
    
    # for each file.
    print(f'Creating {num_files} files.')
    for i in range(num_files):
        filename = f'{data_folder}/ts_file{i}.pkl'
        
        if i % 10 == 0:
            print(f'{filename}')
            
        # get the start and end indices.
        ind0 = i*num_rows_per_file
        ind1 = min(ind0 + num_rows_per_file, end_index)
        data_list = []
        
        # j in the current timestep. Will need j-n to j-1 for the history. And j + target_step for the target.
        for j in range(ind0, ind1):
            indices = range(j-1, j-history_length-1, -step_size)
            data = dataset[sorted(indices) + [j+target_step]]
            
            # append data to the list.
            data_list.append(data)

        df_ts = pd.DataFrame(data=data_list, columns=col_names)
        df_ts.to_pickle(filename)
            
    return len(col_names)-1

Antes de aplicar la función create_ts_files, también necesitamos:

- escalar  *global_active_power* para trabajar con la red.
- definir *n, history_length*, como 7 días (7 * 24 * 60 minutos).
- definir *step_size* dentro de los datos históricos en 10 minutos.
- establecer *target_step* en 10, de modo que estemos pronosticando *global_active_power* 10 minutos después de los datos históricos.

Después de estos, aplicamos *create_ts_files* para:

- Crear 158 archivos (cada uno con un marco de datos de pandas) dentro de la carpeta ts_data.
- Devolver *num_timesteps* como el número de retrasos.

In [ ]:
%%time

global_active_power = df_train['Global_active_power'].values

# Scaled to work with Neural networks.
scaler = MinMaxScaler(feature_range=(0, 1))
global_active_power_scaled = scaler.fit_transform(global_active_power.reshape(-1, 1)).reshape(-1, )

history_length = 7*24*60  # The history length in minutes.
step_size = 10  # The sampling rate of the history. Eg. If step_size = 1, then values from every minute will be in the history.
                #                                       If step size = 10 then values every 10 minutes will be in the history.
target_step = 10  # The time step in the future to predict. Eg. If target_step = 0, then predict the next timestep after the end of the history period.
                  #                                             If target_step = 10 then predict 10 timesteps the next timestep (11 minutes after the end of history).

# The csv creation returns the number of rows and number of features. We need these values below.
num_timesteps = create_ts_files(global_active_power_scaled,
                                start_index=0,
                                end_index=None,
                                history_length=history_length,
                                step_size=step_size,
                                target_step=target_step,
                                num_rows_per_file=128*100,
                                data_folder='ts_data')

# I found that the easiest way to do time series with tensorflow is by creating pandas files with the lagged time steps (eg. x{t-1}, x{t-2}...) and 
# the value to predict y = x{t+n}. We tried doing it using TFRecords, but that API is not very intuitive and lacks working examples for time series.
# The resulting file using these parameters is over 17GB. If history_length is increased, or  step_size is decreased, it could get much bigger.
# Hard to fit into laptop memory, so need to use other means to load the data from the hard drive.


La carpeta ts_data tiene alrededor de 16 GB y solo usamos los últimos 7 días de datos para predecir. ¡Ahora puede ver por qué es necesario dividir el conjunto de datos en dataframes más pequeños!

## Clase TimeSeriesLoader

En este procedimiento, creamos una clase *TimeSeriesLoader* para transformar y alimentar los *dataframe* en el modelo.

Hay funciones integradas de Keras como Keras Sequence, tf.data API. Pero no son muy eficientes para este propósito.



Dentro de esta clase, definimos:

- __init__: la configuración inicial del objeto, que incluye:
- *ts_folder*, que será *ts_data* que acabamos de crear.
- *filename_format*, que es el formato de cadena de los nombres de archivo en *ts_folder*. Por ejemplo, cuando los archivos son *ts_file0.pkl, ts_file1.pkl,…, ts_file100.pkl*, el formato sería "ts_file{}.pkl".
- *num_chunks*: el número total de archivos (fragmentos).
- *get_chunk*: este método toma el dataframe de uno de los archivos y lo procesa para que esté listo para el entrenamiento.
- *shuffle_chunks*: este método baraja el orden de los fragmentos que se devuelven en get_chunk. Esta es una buena práctica para modelar.


Las definiciones pueden parecer un poco confusas. Pero siga leyendo, verá este objeto en acción en el siguiente paso.

In [ ]:
#
# So we can handle loading the data in chunks from the hard drive instead of having to load everything into memory.
# 
# The reason we want to do this is so we can do custom processing on the data that we are feeding into the LSTM.
# LSTM requires a certain shape and it is tricky to get it right.
#
class TimeSeriesLoader:
    def __init__(self, ts_folder, filename_format):
        self.ts_folder = ts_folder
        
        # find the number of files.
        i = 0
        file_found = True
        while file_found:
            filename = self.ts_folder + '/' + filename_format.format(i)
            file_found = os.path.exists(filename)
            if file_found:
                i += 1
                
        self.num_files = i
        self.files_indices = np.arange(self.num_files)
        self.shuffle_chunks()
        
    def num_chunks(self):
        return self.num_files
    
    def get_chunk(self, idx):
        assert (idx >= 0) and (idx < self.num_files)
        
        ind = self.files_indices[idx]
        filename = self.ts_folder + '/' + filename_format.format(ind)
        df_ts = pd.read_pickle(filename)
        num_records = len(df_ts.index)
        
        features = df_ts.drop('y', axis=1).values
        target = df_ts['y'].values
        
        # reshape for input into LSTM. Batch major format.
        features_batchmajor = np.array(features).reshape(num_records, -1, 1)
        return features_batchmajor, target
    
    # this shuffles the order the chunks will be outputted from get_chunk.
    def shuffle_chunks(self):
        np.random.shuffle(self.files_indices)
        

Después de definir, aplicamos este *TimeSeriesLoader* a la carpeta ts_data.

In [ ]:
ts_folder = 'ts_data'
filename_format = 'ts_file{}.pkl'
tss = TimeSeriesLoader(ts_folder, filename_format)

Ahora, con el objeto *tss* apunta a nuestro conjunto de datos, ¡finalmente estamos listos para LSTM!

## Crea el modelo LSTM

La memoria a largo y corto plazo (LSTM) es una arquitectura de red neuronal recurrente artificial (RNN) utilizada en el campo del aprendizaje profundo.

Las redes LSTM son adecuadas para clasificar, procesar y hacer predicciones basadas en datos de series de tiempo, ya que puede haber retrasos de duración desconocida entre eventos importantes en una serie de tiempo.


Vamos a construir un modelo LSTM basado en la API funcional de *tf.keras


Los procedimientos son los siguientes.

- Definir la forma del dataset de entrada:
    - *num_timesteps*, el número de retrasos en los marcos de datos que establecimos en la preparación de los datos.
    - el número de series de tiempo como 1. Ya que solo estamos usando una variable de *global_active_power*.
- Definir el número de unidades, 4 * unidades * (unidades + 2) es el número de parámetros del LSTM. Cuanto mayor sea el número, más parámetros habrá en el modelo.
- Definir el *dropout*, que se utiliza para evitar el sobreajuste.
- Especificar la capa de salida para que tenga una función de activación lineal. 
- Definir el modelo.

### Define el modelo

In [ ]:
# Create the Keras model.
# Use hyperparameter optimization if you have the time.

ts_inputs = tf.keras.Input(shape=(num_timesteps, 1))

# units=10 -> The cell and hidden states will be of dimension 10.
#             The number of parameters that need to be trained = 4*units*(units+2)
x = layers.LSTM(units=10)(ts_inputs)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='linear')(x)
model = tf.keras.Model(inputs=ts_inputs, outputs=outputs)

Luego también definimos la función de optimización y la función de pérdida. Nuevamente, ajustar estos hiperparámetros para encontrar la mejor opción sería una mejor práctica.

### Compila

In [ ]:
# Specify the training configuration.
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mse'])

model.summary()

## Entrenamiento

Entrenamos cada trozo en lotes y solo corremos durante una época. Idealmente, entrenaríamos para múltiples épocas para redes neuronales.

In [ ]:
%%time

# train in batch sizes of 128.
BATCH_SIZE = 128
NUM_EPOCHS = 1
NUM_CHUNKS = tss.num_chunks()

for epoch in range(NUM_EPOCHS):
    print('epoch #{}'.format(epoch))
    for i in range(NUM_CHUNKS):
        X, y = tss.get_chunk(i)
        
        # model.fit does train the model incrementally. ie. Can call multiple times in batches.
        # https://github.com/keras-team/keras/issues/4446
        model.fit(x=X, y=y, batch_size=BATCH_SIZE)
        
    # shuffle the chunks so they're not in the same order next time around.
    tss.shuffle_chunks()

 

## Evaluación del modelo

Al igual que el conjunto de datos de entrenamiento, también creamos una carpeta de los datos de validación, que prepara el conjunto de datos de validación para el ajuste del modelo.

In [ ]:
  # evaluate the model on the validation set.
#
# Create the validation CSV like we did before with the training.
global_active_power_val = df_val['Global_active_power'].values
global_active_power_val_scaled = scaler.transform(global_active_power_val.reshape(-1, 1)).reshape(-1, )

history_length = 7*24*60  # The history length in minutes.
step_size = 10  # The sampling rate of the history. Eg. If step_size = 1, then values from every minute will be in the history.
                #                                       If step size = 10 then values every 10 minutes will be in the history.
target_step = 10  # The time step in the future to predict. Eg. If target_step = 0, then predict the next timestep after the end of the history period.
                  #                                             If target_step = 10 then predict 10 timesteps the next timestep (11 minutes after the end of history).

# The csv creation returns the number of rows and number of features. We need these values below.
num_timesteps = create_ts_files(global_active_power_val_scaled,
                                start_index=0,
                                end_index=None,
                                history_length=history_length,
                                step_size=step_size,
                                target_step=target_step,
                                num_rows_per_file=128*100,
                                data_folder='ts_val_data')

Además de realizar pruebas con el conjunto de datos de validación, también realizamos pruebas con un modelo de línea de base utilizando solo el punto histórico más reciente (t + 10-11).

El código detallado de Python se encuentra a continuación.

In [ ]:
# If we assume that the validation dataset can fit into memory we can do this.
df_val_ts = pd.read_pickle('ts_val_data/ts_file0.pkl')


features = df_val_ts.drop('y', axis=1).values
features_arr = np.array(features)

# reshape for input into LSTM. Batch major format.
num_records = len(df_val_ts.index)
features_batchmajor = features_arr.reshape(num_records, -1, 1)


y_pred = model.predict(features_batchmajor).reshape(-1, )
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1 ,)

y_act = df_val_ts['y'].values
y_act = scaler.inverse_transform(y_act.reshape(-1, 1)).reshape(-1 ,)

print('validation mean squared error: {}'.format(mean_squared_error(y_act, y_pred)))

#baseline
y_pred_baseline = df_val_ts['x_lag11'].values
y_pred_baseline = scaler.inverse_transform(y_pred_baseline.reshape(-1, 1)).reshape(-1 ,)
print('validation baseline mean squared error: {}'.format(mean_squared_error(y_act, y_pred_baseline)))



El conjunto de datos de validación que utiliza LSTM da un error cuadrático medio (MSE) de 0,418. Mientras que el modelo de línea de base tiene un MSE de 0.428. El LSTM lo hace un poco mejor que la línea de base.

Podríamos hacerlo mejor con el ajuste de hiperparámetros y más épocas. 

- [Regresar al inicio](#Contenido)